In [7]:
import HandTrackingModule as htm
from cv2 import cv2
import time
import numpy as np
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [8]:
wCam, hCam = 640, 480

In [9]:
# Hand range      20.5 to 255.0
# Volume range   -37.0 to 1.0

In [10]:
cap = cv2.VideoCapture(1)
cap.set(3, wCam)
cap.set(4, hCam)
pTime = 0

detector =  htm.handDetector(detectionCon=0.8)


devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
#volume.GetMute()
#volume.GetMasterVolumeLevel()
volRange = volume.GetVolumeRange()
#volume.SetMasterVolumeLevel(0.0, None)
minVol = volRange[0]
maxvol = volRange[1]
volBar = 400
volPer = 0

while True:
    success, img = cap.read()
    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw=False)
    if len(lmList) != 0: 
        #print(lmList[4], lmList[8])
        
        x1, y1 = lmList[4][1], lmList[4][2]
        x2, y2 = lmList[8][1], lmList[8][2]
        cx, cy = (x1+x2)//2, (y1+y2)//2
        
        cv2.circle(img, (x1, y1), 12, (255,0,255), cv2.FILLED)
        cv2.circle(img, (x2, y2), 12, (255,0,255), cv2.FILLED)
        cv2.line(img,(x1,y1),(x2,y2),(255,0,255),3)
        cv2.circle(img, (cx, cy), 12, (255,0,255), cv2.FILLED)
        
        length = math.hypot(x2-x1, y2-y1)
        #print(length)
        
        vol = np.interp(length,[20.5,220],[minVol, maxvol])
        volBar = np.interp(length,[20.5,220],[400, 150])
        volPer = np.interp(length,[20.5,220],[0, 100])
        volume.SetMasterVolumeLevel(vol, None)
        #print(int(length),vol)
        
        if length<20.5:
            cv2.circle(img, (cx, cy), 12, (0,255,0), cv2.FILLED)
    
    
    cv2.rectangle(img, (50,150), (85,400), (0,200,0), 3)
    cv2.rectangle(img, (50,int(volBar)), (85,400), (0,200,0), cv2.FILLED)
    cv2.putText(img, f'{int(volPer)} %',(45,450),cv2.FONT_HERSHEY_PLAIN,2,(0,200,0),2)
    
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    cv2.putText(img,f'FPS {int(fps)}',(10,20),cv2.FONT_HERSHEY_PLAIN,1,(255,255,0),1)
    
    cv2.imshow("Img",img)
    cv2.waitKey(1)

error: OpenCV(4.5.2) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-k1ohfcms\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
